# Sentiment analysis of Product reviews using TensorFlow

## Intro:
We all would have used Amazon in our day-to-day life, be it buying groceries, books, spare parts... It can be anything! But have you ever thought how Amazon shows you products, that too only filtered (if you apply them) based on ratings and reviews? Well, there comes sentiment analysis! 

## Definition:
Sentiment analysis (or opinion mining) is a natural language processing (NLP) technique used to determine whether data is positive, negative or neutral. Sentiment analysis is often performed on textual data to help businesses monitor brand and product sentiment in customer feedback, and understand customer needs. Many companies use this technique to improve their products and business.


### Instructions for running this notebook:
If you want to run this notebook, make sure you have GPU for running this program, if no, then you can run this in Kaggle (best because any dataset can be imported directly throught there!) or Google Colab

**Prerequisities needed**:
 - A good dataset (a large review dataset)
 - GPU (TPU is also preferred)
 - Tensorflow Text and Tensorflow Hub (need to install for obtaining the text encoder)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed. This notebook is run on Kaggle
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Library import part

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Installing some libraries as they aren't present in Kaggle or Colab
!pip install -q tensorflow-text
import tensorflow_text as text
import tensorflow_hub as hub

# Below the code, it's a Universal Sentence Encoder where any text can be converted into tensors easily
USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
# Example: How the sentences are equivalent

sent_1 = ["the location is great"]
sent_2 = ["amazing location"]

emb1 = USE(sent_1)
emb2 = USE(sent_2)

print(emb1.shape)
print(np.inner(emb1, emb2).flatten()[0])
# The value will print some equivalent one close to 0.80 to 1

In [ ]:
# Dataset part
dataset = pd.read_csv("../input/consumer-reviews-of-amazon-products/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

dataset.head()
# Prints 1st 5 rows

In [ ]:
# Now, creating two columns for easy view!

dataset["review"] = dataset["reviews.text"]

dataset["score"] = dataset["reviews.rating"].apply(lambda x: "negative" if x<3 else "positive")

# Creating a seperate new variable

df = dataset[["review", "score"]]

In [ ]:
# Testing the reviews....
df.head()

# Saving in seperate CSV file, but it's not important
df.to_csv("results.csv")

In [ ]:
# Viewing values based on segregation we done
df.score.value_counts()

In [ ]:
# Now, seperating values into two diff values
positive_df = df[df["score"] == "positive"]
negative_df = df[df["score"] == "negative"]

print(positive_df.shape, negative_df.shape)

Now, as the data is somewhat unequal, that is, good reviews are more in number than bad ones
So, we're making them in equal to avoid problems!

In [ ]:
good_df = positive_df.sample(n=len(negative_df), random_state=42)
bad_df = negative_df

# This is done to make both data equal in number

In [ ]:
review_df = good_df.append(bad_df).reset_index(drop=True)
review_df.shape

In [ ]:
review_df.head()

After doing that, now we can encode the data and start splitting the dataset

In [ ]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  review_df.score.to_numpy().reshape(-1, 1)
)

In [ ]:
from sklearn.model_selection import train_test_split

train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    review_df.review, 
    type_one_hot, 
    test_size=.1, 
    random_state=42
  )

In [ ]:
from tqdm import tqdm

X_train = []
for r in tqdm(train_reviews):
  emb = USE(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)

X_test = []
for r in tqdm(test_reviews):
  emb = USE(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)

X_test = np.array(X_test)

Here comes the model!

In [ ]:
from keras.utils.vis_utils import plot_model
from keras.layers import LSTM, SpatialDropout1D

model = keras.Sequential()

model.add(
  keras.layers.Dense(
    units=256,
    input_shape=(X_train.shape[1], ),
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)

model.add(
  keras.layers.Dense(
    units=128,
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.5)
)

model.add(
  keras.layers.Dense(
    units=64,
    activation='relu',
    kernel_regularizer='l1' # This one worked so yes
  )
)

model.add(
  keras.layers.Dropout(rate=0.3)
)

model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', 
    optimizer=keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)


print(model.summary())
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(
    X_train, y_train, 
    epochs=22, 
    batch_size=16, 
    validation_split=0.1, 
    verbose=1, 
    shuffle=True
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel("epoch")
plt.ylabel("Cross-entropy loss")
plt.legend();

In [ ]:
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend();

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
# Now, let's test the model with 3 examples
print(test_reviews.iloc[0])
print("Bad" if y_test[0][0] == 1 else "Good")

In [ ]:
y_pred = model.predict(X_test[:1])
print(y_pred)
"Bad" if np.argmax(y_pred) == 0 else "Good"

In [ ]:
print(test_reviews.iloc[10])
print("Bad" if y_test[1][0] == 1 else "Good")

As the model now works correctly, it's time to export it!
Here, we'll be exporting the model using TF Lite/HDF5

In [ ]:
# Saving the model in H5 type
# Type from here!
model.save("Analysis.h5")

The above model can be exported to use directly on Webpage (with help of Flask library in Python)